# Using the Edge Impulse Python SDK with TensorFlow and Keras

<!--- Do not modify the markdown for this example directly! It is generated from a notebook in https://github.com/edgeimpulse/notebooks --->

In [ ]:
# If you have not done so already, install the following dependencies
!python -m pip install tensorflow==2.12.0 edgeimpulse

In [ ]:
from tensorflow import keras
import edgeimpulse as ei

In [ ]:
# Settings
ei.API_KEY = "ei_c90f6349e4f2624274eff30ee840a3e6412c3135d1651ee15515404d290ce842" # Change this to your Edge Impulse API key
labels = ["no fire", "fire"]
num_classes = len(labels)
deploy_filename = "bushfire_detection_model_cpp.zip"

In [ ]:
# Load MNIST data CHANGE
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = keras.utils.normalize(x_train, axis=1)
x_test = keras.utils.normalize(x_test, axis=1)
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
input_shape = x_train[0].shape

In [16]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(num_classes, activation='softmax')  # Softmax for binary classification
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train the model on the bushfire dataset
history = model.fit(
    train_data,
    epochs=10,  # You can adjust the number of epochs
    validation_data=validation_data
)

In [ ]:
# Evaluate model on validation data
score = model.evaluate(validation_data, verbose=0)
print(f"Validation loss: {score[0]}")
print(f"Validation accuracy: {score[1]}")

In [ ]:
# List the available profile target devices
# ei.model.list_profile_devices()

In [ ]:
# Estimate the RAM, ROM, and inference time for our model on the target hardware family
try:
    profile = ei.model.profile(model=model,
                               device='openmv-h7p')
    print(profile.summary())
except Exception as e:
    print(f"Could not profile: {e}")

In [ ]:
# Define the output and input types for your model
labels = ["no_fire", "fire"]  # Update the labels to match your classification task

model_output_type = ei.model.output_type.Classification(labels=labels)
model_input_type = ei.model.input_type.ImageData()  # Assuming images are the input type

# Define the filename for the deployment package
deploy_filename = "bushfire_detection_model.zip"

# Try to deploy the model directly as a C++ library in a .zip file
deploy_bytes = None
try:
    deploy_bytes = ei.model.deploy(
        model=model,  # The trained model
        model_output_type=model_output_type,
        model_input_type=model_input_type,
        deploy_target='zip'  # Export as a .zip file with C++ library
    )
    print("Model deployment successful.")
except Exception as e:
    print(f"Could not deploy: {e}")

# Write the deployed model bytes to a file
if deploy_bytes:
    with open(deploy_filename, 'wb') as f:
        f.write(deploy_bytes.getvalue())
    print(f"Model exported as {deploy_filename}")
else:
    print("Deployment failed; no deployable bytes were generated.")